In [7]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from keras.optimizers import RMSprop

In [8]:
DATASIZE = 1000
SCALE = 10

In [9]:
def getConfig():
    return {
        'inputSize': 1,
        'l1Units' : 8,
        'l2Units' : 8,
        'batchSize': 32}

In [10]:
def getReal():
    data = []

    x = SCALE*(np.random.random_sample((DATASIZE,))-0.5)

    for i in range(DATASIZE):
        yi = 10 + x[i]*x[i]
        data.append([x[i], yi])
    return np.array(data)

In [11]:
def getFake():
    # This is the random input for the generator
    return SCALE*(np.random.random_sample((DATASIZE,))-0.5)

In [12]:
def buildGenerator(cfg):
    nn = tf.keras.models.Sequential()
    nn.add(tf.keras.Input(shape=cfg['inputSize'],))
    nn.add(tf.keras.layers.Dense(cfg['l1Units'], activation='relu'))
    nn.add(tf.keras.layers.Dense(cfg['l2Units'], activation='relu'))
    nn.add(tf.keras.layers.Dense(2))
    return nn

In [14]:
def buildDiscriminator(cfg):
    nn = tf.keras.models.Sequential()
    nn.add(tf.keras.Input(shape=2,))
    nn.add(tf.keras.layers.Dense(cfg['l1Units'], activation='relu'))
    nn.add(tf.keras.layers.Dense(cfg['l2Units'], activation='relu'))
    nn.add(tf.keras.layers.Dense(1))
    return nn

In [15]:
def getBatch(data, cfg):
    idx = np.random.choice(len(data), size=cfg['batchSize'], replace=False)
    return data[idx]

In [16]:
cfg = getConfig()

In [15]:
real = getReal()
fake = getFake()

In [16]:
gen = buildGenerator(cfg)
disc = buildDiscriminator(cfg)
#assert gen.output_shape[1] == data.shape[1], 'invalid shapes'

In [23]:
fakeBatch = getBatch(fake, cfg)
realBatch = getBatch(real, cfg)

##### Loss functions

In [35]:
# Minimize
disc_loss = disc(realBatch) - disc(gen(fakeBatch))
gen_loss = disc(gen(fakeBatch)).numpy().mean()

In [36]:
gen.compile(
    optimizer='rmsprop',
    loss=gen_loss,
    steps_per_execution=1)

In [37]:
gen.loss

0.043404594

In [11]:
''' The discriminator should output very different values for Real vs Fake so
the loss functions will be:
Generator: minimize disc(fake)
Discriminator: maximize disc(actual) - disc(fake)'''

' The discriminator should output very different values for Real vs Fake so\nthe loss functions will be:\nGenerator: minimize disc(fake)\nDiscriminator: maximize disc(actual) - disc(fake)'

In [31]:
class WGANGP():
    def __init__(self, cfg):
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)
        self.generator = self.build_generator(cfg)
        self.critic = self.build_critic(cfg)

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        '''real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)'''
    
    def build_generator(self, cfg):
        nn = tf.keras.models.Sequential()
        nn.add(tf.keras.Input(shape=cfg['inputSize'],))
        nn.add(tf.keras.layers.Dense(cfg['l1Units'], activation='relu'))
        nn.add(tf.keras.layers.Dense(cfg['l2Units'], activation='relu'))
        nn.add(tf.keras.layers.Dense(2))
        return nn
    def build_critic(self, cfg):
        nn = tf.keras.models.Sequential()
        nn.add(tf.keras.Input(shape=2,))
        nn.add(tf.keras.layers.Dense(cfg['l1Units'], activation='relu'))
        nn.add(tf.keras.layers.Dense(cfg['l2Units'], activation='relu'))
        nn.add(tf.keras.layers.Dense(1))
        return nn

In [32]:
cls = WGANGP(cfg)

In [28]:
cls.critic